In [ ]:
!pip install 'transformers[torch]'
!pip install --upgrade openai


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 15.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00


In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
PROJECT_ROOT = f"/content/gdrive/MyDrive/GPT-VLU"
%load_ext autoreload
%autoreload 2

In [ ]:
from transformers import pipeline, set_seed

from utils.setup_utils import parse_and_validate_df, DFExperimentInfo

import pandas as pd
from dataclasses import fields
import openai
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch
import numpy as np

In [ ]:
import sys, os
sys.path.insert(0, PROJECT_ROOT)

In [ ]:
# OPENAI API KEY
openai.api_key = ""

In [ ]:
# Loading data
dataset_path = '/content/gdrive/MyDrive/GPT-VLU/datasets/shape_association.csv'

task = 'choice'
df = parse_and_validate_df(dataset_path, task)
experiment_df = pd.DataFrame(columns=[fields(DFExperimentInfo)])

df.head()

,word,gt,options
0,wedge,triangle,"['triangle', 'rectangle', 'circle']"
1,flag,rectangle,"['triangle', 'rectangle', 'circle']"
2,carrot,triangle,"['triangle', 'rectangle', 'circle']"
3,wheel,circle,"['triangle', 'rectangle', 'circle']"
4,towel,rectangle,"['triangle', 'rectangle', 'circle']"


## Shapes


Possible extra prompts for this:
1. ... instead of MASK, should be 
2. ... 
3. ... fill in the MASK
4. ... replace the MASK with correct word
5. ... choose one option from the list: <> to replace MASK
6. ...

In [ ]:
shapes_orig_prompts = [
    "a photo of a MASK shaped WORD",
    "a photo of a MASK WORD",
    "a photo of the MASK WORD",
    "a MASK WORD",
    "MASK WORD",
    "an image of a MASK WORD",
    "a WORD usually has a MASK shape",
    "WORDs commonly have a MASK shape",
    "the basic shape of a WORD is MASK",
    "what is the shape of a WORD? MASK"
  ]

In [22]:
def score_model(orig_prompts):
  y_true = []
  y_pred = []

  for index, row in tqdm(df.iterrows()):
    w = row['word']
    for p in orig_prompts:
      p = p.replace("WORD", w)

      shapes_options = ['triangle', ' rectangle', ' circle']

      options_results = [] 
      for option in shapes_options:
        input_prompt = p.replace("MASK", option)

        # Extract features from GPT3 
        res = openai.Embedding.create(
          model="text-similarity-davinci-001",
          input=input_prompt,
          max_tokens=3,
        )
        v = torch.tensor(res['data'][0]['embedding']).squeeze().flatten()
        v /= v.norm(p=2, dim=-1, keepdim=True)
        v = v.cpu().numpy()
        options_results.append(v)

      base = torch.tensor(openai.Embedding.create(
          model="text-similarity-davinci-001",
          input=p.replace("MASK", ''),
          max_tokens=3,
        )['data'][0]['embedding']).squeeze().flatten()

      base /= base.norm(p=2, dim=-1, keepdim=True)
      base = base.cpu().numpy()

      scores = np.array(options_results) @ np.expand_dims(base, -1)
      ind_class = np.argmax(scores)
      y_pred.append(shapes_options[ind_class])
      y_true.append(row['gt'])

  return y_true, y_pred


In [24]:
y_true, y_pred = score_model(shapes_orig_prompts)
accuracy_score(y_true, y_pred)

## 1. instead of MASK, should be

In [ ]:
gpt_extra_prompt1 = '. instead of MASK, should be'

In [ ]:
def retrieve_results_options(orig_prompts, max_new=1):
  y_true, y_pred = [], []

  for index, row in df.iloc[1:].iterrows():
    w = row['word']
    gt = row['gt']

    for p in orig_prompts:
      p = p.replace("WORD", w)
      llm_prompt = '. Choose one word from: {} to replace MASK'.format(row['options'])
      input_prompt = p + llm_prompt

      res = openai.Completion.create(
          model="text-davinci-003",
          prompt=input_prompt,
          max_tokens=2 + max_new,
        )
      out_p = res['choices'][0]['text'].replace('\n', '').replace(' ', '').lower()
      y_pred.append(out_p)
      y_true.append(gt)
      
  return y_true, y_pred

In [ ]:
y_true, y_pred = retrieve_results_options(shapes_orig_prompts)
accuracy_score(y_true, y_pred)

0.08703703703703704

In [ ]:
def retrieve_results_nooptions(extra_llm_prompt, orig_prompts, max_new=1):
  y_true, y_pred = [], []

  for index, row in df.iloc[1:].iterrows():
    w = row['word']
    gt = row['gt']

    for p in orig_prompts:
      p = p.replace("WORD", w)
      input_prompt = p + extra_llm_prompt
      res = openai.Completion.create(
          model="text-davinci-003",
          prompt=input_prompt,
          max_tokens=2 + max_new,
        )
      out_p = res['choices'][0]['text'].replace('\n', '').replace(' ', '').lower()
      y_pred.append(out_p)
      y_true.append(gt)
      
  return y_true, y_pred

In [ ]:
y_true, y_pred = retrieve_results_nooptions('. instead of MASK should be', shapes_orig_prompts, max_new=1)
accuracy_score(y_true, y_pred)

0.025

In [ ]:
y_true, y_pred = retrieve_results_nooptions('. MASK can be replaced with', shapes_orig_prompts, max_new=1)
accuracy_score(y_true, y_pred)

0.000925925925925926

In [ ]:
y_true, y_pred = retrieve_results_nooptions('. Fill in the MASK: ', shapes_orig_prompts, max_new=1)
accuracy_score(y_true, y_pred)

0.002777777777777778